In [2]:
import os
import pandas as pd

# Directories
converted_dir = r"output\html_xlsx_converted"
cleaned_dir = r"output\cleaned_excels"
os.makedirs(cleaned_dir, exist_ok=True)

# Known states
states_uts = [
    "Andaman and Nicobar", "Andhra Pradesh", "Arunachal pradesh", "Assam", "Bihar", "Chandigarh", "Chhattisgarh",
    "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Jharkhand", "Karnataka", "Kerala",
    "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Odisha", "Puducherry", "Punjab", "Rajasthan",
    "Sikkim", "Tamil Nadu", "Telangana", "Tripura", "Uttar Pradesh", "West Bengal"
]

# Final column names
columns_required = [
    "Date", "States/UTs", "Rice", "Wheat", "Atta (Wheat)", "Gram Dal", "Tur/Arhar Dal", "Urad Dal", "Moong Dal", "Masoor Dal",
    "Sugar", "Milk @", "Groundnut Oil (Packed)", "Mustard Oil (Packed)", "Vanaspati (Packed)", "Soya Oil (Packed)",
    "Sunflower Oil (Packed)", "Palm Oil (Packed)", "Gur", "Tea Loose", "Salt Pack (Iodised)", "Potato", "Onion", "Tomato"
]

def clean_xlsx_and_cast(file_path, output_dir):
    try:
        filename = os.path.basename(file_path)
        date_str = filename.split("_")[1]  # e.g., 20200201
        date_formatted = f"{date_str[6:8]}-{date_str[4:6]}-{date_str[:4]}"  # new: DD-MM-YYYY

        df = pd.read_excel(file_path, header=None)
        df = df.dropna(how='all').dropna(axis=1, how='all')

        # Remove junk rows like "Average Price", etc.
        junk_labels = ["Average Price", "Maximum Price", "Minimum Price", "Modal Price"]
        df = df[~df.apply(lambda row: any(str(cell).strip() in junk_labels for cell in row), axis=1)]

        # Retain only rows with valid state name (case insensitive match)
        states_lower = [s.lower() for s in states_uts]
        df = df[df.apply(lambda row: any(str(cell).strip().lower() in states_lower for cell in row), axis=1)]

        print(f"🔎 Preview {filename} → {df.shape[0]} rows retained before fix")

        # Ensure exactly one state per row, by finding the column that has the state name
        def extract_state_and_fix_row(row):
            for i, cell in enumerate(row):
                if str(cell).strip().lower() in states_lower:
                    state = str(cell).strip()
                    rest = list(row)
                    rest[i] = None  # remove state from that position
                    return [date_formatted, state] + rest[:i] + rest[i+1:]
            return None

        fixed_rows = [extract_state_and_fix_row(row) for _, row in df.iterrows()]
        fixed_rows = [r for r in fixed_rows if r is not None]

        # Create new DataFrame with cleaned structure
        cleaned_df = pd.DataFrame(fixed_rows)

        # Truncate or pad to 24 columns
        cleaned_df = cleaned_df.iloc[:, :24]
        cleaned_df.columns = columns_required[:cleaned_df.shape[1]]

        # Convert price columns to float
        for col in cleaned_df.columns[2:]:
            cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce')

        # Save cleaned file
        out_file = os.path.join(output_dir, filename.replace(".xlsx", "_cleaned.xlsx"))
        cleaned_df.to_excel(out_file, index=False)
        print(f"✅ Cleaned: {file_path}")
    except Exception as e:
        print(f"❌ Failed to clean {file_path}: {e}")

# Run cleaning on all converted files
for file in os.listdir(converted_dir):
    if file.endswith(".xlsx"):
        clean_xlsx_and_cast(os.path.join(converted_dir, file), cleaned_dir)


🔎 Preview excel_20200201_Report_Data_Commoditywise_with_variation (1).xlsx → 26 rows retained before fix
✅ Cleaned: output\html_xlsx_converted\excel_20200201_Report_Data_Commoditywise_with_variation (1).xlsx
🔎 Preview excel_20200202_Report_Data_Commoditywise_with_variation.xlsx → 30 rows retained before fix
✅ Cleaned: output\html_xlsx_converted\excel_20200202_Report_Data_Commoditywise_with_variation.xlsx
🔎 Preview excel_20200203_Report_Data_Commoditywise_with_variation (1).xlsx → 31 rows retained before fix
✅ Cleaned: output\html_xlsx_converted\excel_20200203_Report_Data_Commoditywise_with_variation (1).xlsx
🔎 Preview excel_20200204_Report_Data_Commoditywise_with_variation.xlsx → 18 rows retained before fix
✅ Cleaned: output\html_xlsx_converted\excel_20200204_Report_Data_Commoditywise_with_variation.xlsx
🔎 Preview excel_20200205_Report_Data_Commoditywise_with_variation (1).xlsx → 30 rows retained before fix
✅ Cleaned: output\html_xlsx_converted\excel_20200205_Report_Data_Commoditywise_